In [1]:
from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize
import re
from elasticsearch import Elasticsearch
import os
from io import StringIO
from io import BytesIO
import json
import pandas as pd
import sklearn
import numpy as np


In [2]:
from sklearn_pandas import DataFrameMapper, cross_val_score

In [3]:
import pandas as pd
import numpy as np
import sklearn.preprocessing, sklearn.decomposition, \
    sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import glob


path =r'C:\Users\tlang\OneDrive\Documents\tzviya work\mars\converting users' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0,low_memory=False)
    list_.append(df)
frame = pd.concat(list_)

#df=frame[:100]

In [26]:
df=frame[:10000]

In [5]:
df=frame

In [6]:
df.dtypes

scenario_rule_value         int64
app_bundle                 object
device_ifa                 object
hour_dim                    int64
day_dim                     int64
publisher_id_dim            int64
connection_type_dim       float64
country_dim                object
city_dim                  float64
carrier_dim               float64
user_gender_dim            object
user_yob_dim              float64
device_os_dim              object
agent_id_dim                int64
valid_device_ifa_dim        int64
imp_pub_media_dim          object
imp_pub_plc_dim            object
imp_pub_cmp_dim            object
store_app_cat_dim          object
desc_regex_app_cat_dim     object
loops_amount_dim           object
converted                   int64
dtype: object

In [7]:

clicks_per_app=df.groupby('app_bundle')['app_bundle'].count().reset_index(name="app_clicks")
conversions_per_app=df.groupby('app_bundle')['converted'].sum().reset_index(name="app_conversions")

ratingdf=pd.merge(clicks_per_app,conversions_per_app,on='app_bundle')

ratingdf['app_rating']=100*ratingdf['app_conversions']/ratingdf['app_clicks']
#ratingdf=ratingdf[['app_bundle','app_rating']]
extframe =pd.merge(df,ratingdf,on='app_bundle')

clicks_per_user=df.groupby('device_ifa')['device_ifa'].count().reset_index(name="clicks")
conversions_per_user=df.groupby('device_ifa')['converted'].sum().reset_index(name="conversions")

consumptiondf=pd.merge(clicks_per_user,conversions_per_user,on='device_ifa')

consumptiondf['user_consumption']=100*consumptiondf['conversions']/consumptiondf['clicks']
consumptiondf=consumptiondf[['device_ifa','user_consumption']]
extframe =pd.merge(extframe,consumptiondf,on='device_ifa')


In [8]:
frame.shape

(13498904, 22)

In [ ]:
extframe

In [40]:
extframe=extframe.drop(['device_ifa', 'desc_regex_app_cat_dim','loops_amount_dim'],  axis=1)

In [43]:
extframe.dtypes

scenario_rule_value       int64
app_bundle               object
hour_dim                  int64
day_dim                   int64
publisher_id_dim          int64
connection_type_dim     float64
country_dim              object
city_dim                float64
carrier_dim             float64
user_gender_dim          object
user_yob_dim            float64
device_os_dim            object
agent_id_dim              int64
valid_device_ifa_dim      int64
imp_pub_media_dim        object
imp_pub_plc_dim          object
imp_pub_cmp_dim          object
store_app_cat_dim        object
converted                 int64
app_clicks                int64
app_conversions           int64
app_rating              float64
user_consumption        float64
dtype: object

In [47]:
df=extframe

cat_lists=df['store_app_cat_dim'].unique()
cat_lists=[l.split(',') for l in cat_lists if type(l) is str]
flat_cats = [item for sublist in cat_lists for item in sublist]

app_cats=list(set(flat_cats))
    

for cat in app_cats:
    df[cat]= df['store_app_cat_dim'].apply(lambda x: 1 if type(x) is str and cat in x else 0)

In [53]:
df.iloc[0]

scenario_rule_value                             31060862
app_bundle                                   id489472613
hour_dim                                               8
day_dim                                                1
publisher_id_dim                                   11083
connection_type_dim                                    2
country_dim                                           FR
city_dim                                             NaN
carrier_dim                                        25117
user_gender_dim                                      NaN
user_yob_dim                                         NaN
device_os_dim                                     iOS 11
agent_id_dim                                        3819
valid_device_ifa_dim                                   0
imp_pub_media_dim       2ff93175c65b4ea4ad4ed56e40196550
imp_pub_plc_dim         2ff93175c65b4ea4ad4ed56e40196550
imp_pub_cmp_dim                                   208-26
store_app_cat_dim              

In [55]:
df=df.drop(['store_app_cat_dim'],  axis=1)

In [56]:
#import sklearn2pmml 
from sklearn2pmml.decoration import CategoricalDomain,ContinuousDomain
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
joined=df

mapper = DataFrameMapper([(i, ContinuousDomain()) if j != 'object' and j != 'bool' else (i,
                                               [CategoricalDomain(missing_value_treatment = "as_value",
                                                invalid_value_treatment = "as_missing",
                                                missing_value_replacement = joined[i].value_counts().idxmax()),
                                               preprocessing.LabelEncoder()])
                                  for i, j in
                                  zip(joined.columns[:-1].values, joined.dtypes[:-1].values)], 
                                  input_df=True, df_out=True)
np.round(mapper.fit_transform(df.copy()), 2)

C:\Users\tlang\Anaconda3\lib\site-packages\sklearn2pmml\decoration\__init__.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X[mask] = self.missing_value_replacement


,scenario_rule_value,app_bundle,hour_dim,day_dim,publisher_id_dim,connection_type_dim,country_dim,city_dim,carrier_dim,user_gender_dim,...,finance,business,game_strategy,education,game_music,game_family,weather,news_and_magazines,game_card,game_racing
0,31060862.0,202.0,8.0,1.0,11083.0,2.0,8.0,NaN,25117.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31060862.0,202.0,6.0,3.0,10720.0,2.0,8.0,2973783.0,25117.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38646359.0,202.0,0.0,7.0,10670.0,2.0,1.0,2063523.0,133612.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,56992245.0,168.0,5.0,3.0,10670.0,2.0,1.0,2063523.0,133612.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,52643202.0,202.0,9.0,2.0,10731.0,2.0,5.0,2885408.0,6805.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,60852882.0,131.0,7.0,5.0,10731.0,2.0,5.0,2825297.0,6805.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,31060869.0,262.0,7.0,7.0,10731.0,2.0,5.0,2949012.0,6805.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7,48886382.0,187.0,8.0,3.0,10731.0,2.0,5.0,2885408.0,6805.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,31060862.0,202.0,5.0,7.0,10720.0,2.0,8.0,2973783.0,25117.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,31060862.0,202.0,5.0,5.0,10720.0,2.0,8.0,2973783.0,25117.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df.groupby('imp_pub_plc_dim').count()

,scenario_rule_value,app_bundle,device_ifa,hour_dim,day_dim,publisher_id_dim,connection_type_dim,country_dim,city_dim,carrier_dim,...,user_yob_dim,device_os_dim,agent_id_dim,valid_device_ifa_dim,imp_pub_media_dim,imp_pub_cmp_dim,store_app_cat_dim,desc_regex_app_cat_dim,loops_amount_dim,converted
imp_pub_plc_dim,,,,,,,,,,,,,,,,,,,,,
Clock,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,0,1,2,2,2
Dating,12,12,12,12,12,12,12,12,12,12,...,5,12,12,12,12,0,12,12,12,12
Mixtape App,2069,2069,2069,2069,2069,2069,2069,2069,2001,2069,...,770,2069,2069,2069,2069,0,2061,2069,2069,2069
Stock Market Tracker,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,0,2,2,2,2
0018a2dfe6454d4eacf878cd7acadf85,6063,6063,6063,6063,6063,6063,6063,6063,5655,6063,...,5575,6063,6063,6063,6063,4907,6019,6063,6063,6063
0018a5d677ea42e581c4d2a0c8bc84a9,10,10,10,10,10,10,10,10,7,10,...,3,10,10,10,10,7,10,10,10,10
002cd51568af4da28f5a2cb593d4250c: IsarInteractive_SkatFREE_iOS,15,15,15,15,15,15,15,15,15,15,...,0,15,15,15,15,0,15,15,15,15
0038a266cbbc418b84fb32c231d88933,17,17,17,17,17,17,17,17,17,17,...,0,17,17,17,17,17,17,17,17,17
003d669296ed484eba2a1e2dc435e0ad,9802,9788,9802,9802,9802,9802,9802,9802,9182,9802,...,6091,9802,9802,9802,9788,5913,9672,9787,9802,9802


In [13]:
len(df['imp_pub_plc_dim'].value_counts())

4310

In [14]:
df.shape

(13498904, 22)